### Installation

### **Clone and install the Tensorflow Object Detection API** 

In order to use the TensorFlow Object Detection API, we need to clone it's GitHub Repo.
<br>

#### **Dependencies**


Most of the dependencies required come preloaded in Google Colab.  No extra installation is needed.
<br>

#### **Protocol Buffers**



The TensorFlow Object Detection API relies on what are called `protocol buffers` (also known as `protobufs`). Protobufs are a language neutral way to describe information. That means you can write a protobuf once and then compile it to be used with other languages, like Python, Java or C [5].

The `protoc` command used below is compiling all the protocol buffers in the `object_detection/protos` folder for Python.

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

#### Uninstall and reinstall OpenCV
This is required to make the notebook compatible with earlier version of TensorFlow.

In [ ]:
pip uninstall opencv-python-headless==4.5.5.62

In [ ]:
pip install opencv-python-headless==4.5.2.52

#### Restart Runtime.
We need to restart runtime. Click on **Runtime** > click **Restart runtime**.

#### Uninstall and reinstall TensorFlow 2.8
Again, this is required to make the notebook compatible for model training. If this is not done, `DNN library is not found error` will arise.

In [ ]:
!pip install tensorflow==2.8

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Run the model builder test

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

---

### Getting the data

In [ ]:
!pip install -q kaggle
!pip install -q kaggle-cli

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = '<your-kaggle-username>'
os.environ['KAGGLE_KEY'] = '<your-kaggle-key>'

In [ ]:
%%bash
mkdir /content/dataset
cd /content/dataset
kaggle datasets download -d nstiwari/graph-detection --unzip

##### Create a `labelmap.pbtxt` file in /content directory. Take the following as an example:



```
item {
  name: "pie_chart"
  id: 1
}

item {
  name: "donut"
  id: 2
}

item {
  name: "bar_graph"
  id: 3
}

item {
  name: "line_chart"
  id: 4
}

item {
  name: "area_chart"
  id: 5
}
```



##### Converting data to TFRecord
The dataset contains two files `train_labels.csv` and `test_labels.csv` which need to be converted into TFRecord format so that it can be fed into Tensorflow’s 2 Object Detection API.

In [ ]:
%cd /content/

In [ ]:
!wget https://raw.githubusercontent.com/NSTiwari/Visual-Data-Extractor-using-TensorFlow/master/generate_tf_records.py

In [ ]:
!wget https://raw.githubusercontent.com/NSTiwari/Visual-Data-Extractor-using-TensorFlow/master/xml_to_csv.py -P /content/dataset/graph-detection/

In [ ]:
%cd /content/
!python dataset/graph-detection/xml_to_csv.py

In [ ]:
!python generate_tf_records.py -l labelmap.pbtxt -o dataset/train.record -i dataset/graph-detection/images -csv dataset/graph-detection/train_labels.csv
!python generate_tf_records.py -l labelmap.pbtxt -o dataset/test.record -i dataset/graph-detection/images -csv dataset/graph-detection/val_labels.csv

---

### Configuring train

We are going to use the pretrained TF2 MobileNet V2 model as the feature extractor in the SSD MobileNet V2 Object Detection model. So the next logical step is to download and untar the pretrained TF2 MobileNet V2 model.

#### Downloading MobileNet weights

In [ ]:
%cd /content
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

#### Defining training parameters

In [ ]:
num_classes = 5
batch_size = 24 #16
num_steps = 7500 #1500
num_eval_steps = 1000

train_record_path = '/content/dataset/train.record'
test_record_path = '/content/dataset/test.record'
model_dir = '/content/training/'
labelmap_path = '/content/labelmap.pbtxt'

pipeline_config_path = 'mobilenet_v2.config'
fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'

#### Editing config file

The next cell is modification of the code available at [4].

In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
  'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint),
                  config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  f.write(config)

---

### Training the Network

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

### Export the Inference Graph

The below code cell adds a line to the tf_utils.py file. This is a temporary fix to a exporting issue occuring when using the OD API with Tensorflow 2. This code will be removed as soon as the OD Team puts out a fix.

All credit goes to the Github users [Jacobsolawetz](https://github.com/Jacobsolawetz) and [ Tanner Gilbert](https://github.com/TannerGilbert), who provided this [temporary fix](https://github.com/tensorflow/models/issues/8841#issuecomment-657647648).

In [ ]:
with open('/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/tf_utils.py') as f:
    tf_utils = f.read()

with open('/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/tf_utils.py', 'w') as f:
  # Set labelmap path
  throw_statement = "raise TypeError('Expected Operation, Variable, or Tensor, got ' + str(x))"
  tf_utils = tf_utils.replace(throw_statement, "if not isinstance(x, str):" + throw_statement)
  f.write(tf_utils)

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

##### Downloading weights

In [ ]:
!zip -r /content/saved_model.zip /content/inference_graph/saved_model/

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")